# Backcasting Notebook

_Loren Champlin_

Adapted from _Adarsh Pyarelal_'s WM 12 Month Evaluation Notebook 

As always, we begin with imports, and print out the commit hash for a rendered
version of the notebook.

In [ ]:
%load_ext autoreload
%autoreload 2
%matplotlib inline
import pickle
from IPython.display import set_matplotlib_formats
set_matplotlib_formats('retina')
from delphi.visualization import visualize
import delphi.jupyter_tools as jt
import numpy as np
import pandas as pd
from delphi.db import engine
jt.print_commit_hash_message()
import random as rm

Here I will set random seeds

In [ ]:
np.random.seed(87)
rm.seed(87)

In [ ]:
with open("../scripts/build/quantified_CAG.pkl", "rb") as f:
    G = pickle.load(f)

In the cell below, we visualize the CAG.

Legend: 
- Red edge: overall inhibition, green edge: overall promotion
- Edge thickness corresponds roughly to the 'strength' of the influence.
- Edge opacity corresponds roughly to the number of evidence fragments 
  that support the causal relationship.

In [ ]:
visualize(G, indicators=True, indicator_values=True)

Lists all nodes in the CAG, good for seeing the full name of each node. 

In [ ]:
for n in G.nodes():
    print(n)

"intervene" is the node whose attached indicator variable we would like to apply an intervention. 

"delt" is the change applied to the attached indicator variable per time step (ex: -0.2, means that it decreases by 20% per time step). 

"t_step" is how many timesteps we are taking. 

"target" is the node whose indicator variable we would like to predict. Of course the "run_experiment" method provides a plot for all indicator variables, but I am currently working on obtaining an estimate and confidence interval for the just the target variable. 

"est" is the chosen estimator for the group of samples generated by "run_experiment". This must be a callable function. 

"conf" is the confidence interval for "est".

"target_indicator" is the indicator variable name attach to the target node (we assume there is only one for now).

"intervener_indicator" is the indicator variable name attach to the node we are intervening on. 

In [ ]:
intervene = "UN/entities/human/financial/economic/revenue"
delt = 0.0159574468
t_step = 60
target = "UN/events/human/human_migration"
est = np.median
conf = 68
target_indicator = list(G.nodes(data=True)[target]['indicators'].keys())[0]
intervener_indicator = list(G.nodes(data=True)[intervene]['indicators'].keys())[0]

Retrieves mean value of the initial indicator value for the indicator variable attached to the target node.  

In [ ]:
list(G.nodes()[target]['indicators'].values())[0].mean

Estimates a chosen estimator (median by default) along with a confidence interval (68% by default) for each time step for each indicator variable in the above CAG. These estimated values are plotted as a line and the confidence interval is a shaded region. 

As of now, 200 samples are generated at each time step. 

In [ ]:
jt.run_experiment(G, intervene, delt, t_step,est=est,conf = conf)

Median of the 200 samples at the final timestep for the indicator variable attached to the target node. 

In [ ]:
np.median(list(G.nodes(data=True)[target]['indicators'].values())[0].samples)

In [ ]:
np.median(list(G.nodes()[intervene]['indicators'].values())[0].samples)

In [ ]:
engine.table_names()

In [ ]:
df = pd.read_sql_table("indicator", con=engine)
df[df['Variable'] == intervener_indicator]


In [ ]:
df[df['Variable'] == target_indicator]